# Preperation

In [1]:
!pip install transformers
!gdown --folder "1cszZtjGiWoS5kJEU3cF-VZaXDYMiV7KR"

Retrieving folder list
Processing file 1ZzikX7CWHoFFHMZKEclZSuusKMy6JgfD bias-filtered-reduced.csv
Processing file 1kPk4UsXsTxHkAjnklxbbLsSQqM2ZikqP bias-filtered.csv
Processing file 1OIKLoXWjU2PX7-53E6bwcZjzlzB8I3u7 bias-neutrality.csv
Processing file 1VZOwfmloJxDUXTZgk_uzt_fm5U56QPCa netr-filtered-reduced.csv
Processing file 1VuiJdcgtDInrlKDdqGABoIR4Luib6LJc netr-filtered.csv
Processing file 1MeRfyhEXB1BhW4JK_VPTrcqWEKuGnknC subjectivity-MPQA-All-News.csv
Processing file 1egTv57Du45zPTS17DklrJgisRoG6Uwe2 subjectivity-MPQA-All.csv
Processing file 1XZ4ulTvgN3yxWuEZG8BqTrhJyh2c62bT subjectivity-Rotten IMDB.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1ZzikX7CWHoFFHMZKEclZSuusKMy6JgfD
To: /content/Clean Dataset/bias-filtered-reduced.csv
100% 230k/230k [00:00<00:00, 89.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kPk4UsXsTxHkAjnklxbbLsSQqM2ZikqP
To: /content/Clean

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import keras
from keras import layers, metrics
import pandas as pd
import numpy as np
import math
import matplotlib as plt
from google.colab import files
import shutil, os

# Load Dataset

## Function & Global Variables

In [4]:
TRAIN_SIZE = 0.6
VAL_SIZE = 0.2
TEST_SIZE = 0.2
SUBJ_BATCH_SIZE = 4

bert_base = 'indolem/indobert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_base)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

In [5]:
def encode(text):
    encoded_dict = tokenizer(text, add_special_tokens = True, max_length = 512,
                             padding='max_length', return_attention_mask = True,
                             truncation=True, return_tensors = 'tf')

    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return (input_ids, attention_masks)

In [6]:
def split_input_labels(features, labels):
    return {'input_ids': features[0], 'attention_mask': features[1]}, labels

In [7]:
def split_dataset(dataset):
  train_val_dataset, test_dataset = tf.keras.utils.split_dataset(
    dataset, left_size=TRAIN_SIZE+VAL_SIZE, right_size=TEST_SIZE, shuffle=True, seed=42
  )

  train_dataset, val_dataset = tf.keras.utils.split_dataset(
      train_val_dataset, left_size=TRAIN_SIZE, right_size=VAL_SIZE, shuffle=True, seed=42
  )

  return (train_dataset.map(split_input_labels),
          val_dataset.map(split_input_labels),
          test_dataset.map(split_input_labels))

In [8]:
def dataset_preperation(text, labels, batch_size):
  features_tensor = encode(text.values.tolist())
  labels_tensor = tf.constant(labels, dtype=tf.int32)
  dataset = tf.data.Dataset.from_tensor_slices((features_tensor, labels_tensor))

  train, val, test = split_dataset(dataset)

  return (train.batch(batch_size),
          val.batch(batch_size),
          test.batch(batch_size))

## Subjectivity

In [9]:
df_subjectivity = pd.read_csv('/content/Clean Dataset/subjectivity-MPQA-All-News.csv')
features = df_subjectivity['content']
labels = df_subjectivity['is_subjective']

In [10]:
train_subjectivity, val_subjectivity, test_subjectivity = dataset_preperation(features, labels, SUBJ_BATCH_SIZE)

# Modeling

## Subjectivity

In [11]:
def create_subjectivity_model(max_len):
  input_ids = layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
  attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

  bert = TFBertModel.from_pretrained(bert_base, from_pt=True).bert(input_ids, attention_mask)

  classifier = keras.Sequential([
        layers.Dense(256, input_shape=(768,), activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation='sigmoid')
  ])(bert.pooler_output)

  model = keras.Model(inputs=[input_ids, attention_mask],
                     outputs=classifier)

  optimizer = tf.keras.optimizers.Adam(epsilon=1)

  model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

  return model

In [12]:
SUBJ_EPOCHS = 10
SUBJ_PATIENCE = 3
SUBJ_VERSION = 2

In [16]:
subjectivity_model = create_subjectivity_model(512)
subjectivity_model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predict

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 512)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1105582   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   08         'attention_mask[0][0]']      
                             hidden_state=(None, 512, 7                                     

In [17]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=SUBJ_PATIENCE,
                                                  restore_best_weights=True)

subjectivity_model.fit(train_subjectivity, epochs=SUBJ_EPOCHS, validation_data = val_subjectivity,
                       callbacks=[early_stopping])

Epoch 1/10
128/128 [==============================] - 133s 653ms/step - loss: 0.6929 - accuracy: 0.5285 - val_loss: 0.6712 - val_accuracy: 0.5824
Epoch 2/10
128/128 [==============================] - 81s 631ms/step - loss: 0.6724 - accuracy: 0.5796 - val_loss: 0.6462 - val_accuracy: 0.6647
Epoch 3/10
128/128 [==============================] - 78s 606ms/step - loss: 0.6422 - accuracy: 0.6306 - val_loss: 0.6180 - val_accuracy: 0.6824
Epoch 4/10
128/128 [==============================] - 80s 625ms/step - loss: 0.6085 - accuracy: 0.6640 - val_loss: 0.5792 - val_accuracy: 0.7647
Epoch 5/10
128/128 [==============================] - 80s 624ms/step - loss: 0.5633 - accuracy: 0.7250 - val_loss: 0.5583 - val_accuracy: 0.7471
Epoch 6/10
128/128 [==============================] - 80s 625ms/step - loss: 0.5220 - accuracy: 0.7485 - val_loss: 0.5333 - val_accuracy: 0.7294
Epoch 7/10
128/128 [==============================] - 78s 607ms/step - loss: 0.4696 - accuracy: 0.7976 - val_loss: 0.5231 - val_a

In [18]:
subjectivity_model.predict(test_subjectivity)

53/53 [==============================] - 13s 181ms/step


array([[0.04840527],
       [0.03583318],
       [0.88504213],
       [0.75044227],
       [0.49516898],
       [0.06306139],
       [0.03253641],
       [0.5312482 ],
       [0.14875019],
       [0.7665318 ],
       [0.02495556],
       [0.04620096],
       [0.04620096],
       [0.8036599 ],
       [0.6034698 ],
       [0.03847224],
       [0.74973774],
       [0.40361157],
       [0.15942574],
       [0.8108406 ],
       [0.03566296],
       [0.6390504 ],
       [0.05363122],
       [0.7984406 ],
       [0.03734328],
       [0.87672   ],
       [0.13941787],
       [0.84980285],
       [0.02750407],
       [0.4425359 ],
       [0.84440535],
       [0.07547132],
       [0.2101797 ],
       [0.1948409 ],
       [0.03826368],
       [0.12106157],
       [0.43505442],
       [0.23417394],
       [0.03826368],
       [0.21254076],
       [0.04613626],
       [0.05607771],
       [0.04192298],
       [0.08139759],
       [0.78249395],
       [0.42123857],
       [0.75581145],
       [0.077

In [19]:
subjectivity_model.evaluate(test_subjectivity)

53/53 [==============================] - 10s 188ms/step - loss: 0.5025 - accuracy: 0.7311


[0.5025366544723511, 0.7311320900917053]

In [20]:
subjectivity_model.save(f'/content/drive/MyDrive/Capstone/Model/Subjectivity/SV{SUBJ_VERSION}')